All groups and individual must do the following:

    Convert the app sizes to a number
    Convert the number of installs to a number
    Transform “Varies with device” into a missing value
    Convert Current Ver and Android Ver into a dotted number (e.g. 4.0.3 or 4.2)
    Remove the duplicates
    For each category, compute the number of apps
    For each category, compute the average rating
    Create two dataframes: one for the genres and one bridging apps and genres. So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity
    For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)
    For each genre, compute the average rating. What is the genre with highest average?
    For each app, compute the approximate income, obtain as a product of number of installs and price.
    For each app, compute its minimum and maximum Sentiment_polarity

The following part of the exercise must be done only by groups with two or three people

    For each app, compute the average number of words in its reviews
    For each app, compute its longest review
    For each app, compute the ratio between the number of installs and the number of reviews
    Cluster the apps according to the major android version (the first two digits — e.g. for 4.0.3 the major version is 4.0)
    For each cluster, compute the average date and the last date of an update.
    Excluding the free apps, what is the content rating with highest average price?

The following part of the exercise must be done only by groups with three people

    What is the genre with the highest total income?
    What is the genre with the highest fraction of free apps (over the number of all apps)?
    For each rating, compute the average income
    For each (Content Rating, Genre) pair, compute the number of reviews and the average rating.


In [99]:
#librerie
import pandas as pd
import numpy as np
import re
from datetime import datetime
from dateutil.parser import parse

In [110]:
gps = pd.read_csv('googleplaystore.csv', low_memory = False)
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [111]:
#SE Android_ver manca levare riga, se doppia considerare solo ultima
#Malformed row levare (quella con Free)

#Sistemiamo il dataset eliminando la riga malformata (10472) e le righe
#in cui manca la versione di android
gps = gps.drop(gps.index[10472])
gps = gps[gps['Android Ver'].notnull()]
gps = gps[gps['Android Ver'] != 'Varies with device']

#castiamo a numeric l'attributo Reviews
gps['Reviews'] = pd.to_numeric(gps['Reviews'])

In [55]:
#Convert the app sizes to a number
#Transform “Varies with device” into a missing value

#definizione del pattern regex per parsare l'attributo Size
string_to_num_size = re.compile('(?P<val>\d+\.{0,1}\d*)(?P<unit>[\w{0,1}|\+{0,1}])')

unit_to_val = {'M' : 1000000,
               'k' : 1000,
               '+' : 1000,
               'Varies with device': np.nan}

def size_to_num(value):
    pippo = string_to_num_size.search(value.replace(",", "."))
    if(pippo != None):
        num = float(pippo.group('val'))
        return int(num)*unit_to_val[pippo.group('unit')]
    else:
        return unit_to_val[value]

In [56]:
#Convert the number of installs to a number
#C'è una riga con installs = Free
string_to_num_installs = re.compile('(?P<val>\d+)(?P<plus>\+{0,1})')

def installs_to_num(value):
    pluto = string_to_num_installs.search(value.replace(",",""))
    if(pluto != None):
        return int(pluto.group('val'))
    else:
        return np.nan

In [112]:
gps['IntSize'] = gps['Size'].apply(size_to_num)
gps['IntInstalls'] = gps['Installs'].apply(installs_to_num)
gps['Last Updated date'] = gps['Last Updated'].apply(pd.to_datetime) #Conversione da String a Datetime attributo Last Updated
gps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,IntSize,IntInstalls,Last Updated date
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,19000000.0,10000,2018-01-07
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,14000000.0,500000,2018-01-15
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,8000000.0,5000000,2018-08-01
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,25000000.0,50000000,2018-06-08
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2000000.0,100000,2018-06-20


In [113]:
#Remove the duplicates

#controlliamo prima se esitono delle righe totalmente identiche da eliminare
print(gps.shape)
gps = gps.drop_duplicates()
print(gps.shape)

(9476, 16)
(9134, 16)


In [19]:
gps[gps['App'] == 'Pixel Draw - Number Art Coloring Book']
#gps['App'].unique()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,IntSize,IntInstalls
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,2000000.0,100000


In [115]:
def remove_duplicate_rows(old_df):
    new_df = pd.DataFrame(columns=old_df.columns)
    app_name = old_df['App'].unique()
    for app in app_name:
        #slice del df solo con righe app presa in considerazione
        tmp_df = old_df[old_df['App'] == app]
        #se la lunghezza del df tmp è pari a 1 non controllo, c'è solo 1 app
        if(tmp_df.shape[0] != 1):
            #devo controllare se effetivamente sono le stesse o meno
            #
            # Prima controllo che coppia nome - last update sia unica, tengo solo più recente
            # f.groupby(['A','C'], group_keys=False).apply(lambda x: x.ix[x.B.idxmax()]).reset_index(drop=True)
            #
            # nome inutile controllare, size (?), type, price, genres, tengo quella con n°reviews più alto
            # Se ho solo 1 unico valore in questi attributi allora sono la stessa e tengo quella con max reviews
            # 
            tmp_df = tmp_df.groupby(['App','Current Ver', 'Android Ver', 'Last Updated date', 'Genres'], group_keys=False).apply(lambda x: x.loc[x['Reviews'].idxmax()]).reset_index(drop=True)
            print(tmp_df)
            if(len(tmp_df['Size'].unique()) == 1 and len(tmp_df['Type'].unique()) == 1 and
               len(tmp_df['Price'].unique()) == 1 and len(tmp_df['Genres'].unique()) == 1):
                new_df = new_df.append(tmp_df.loc[tmp_df['Reviews'].idxmax()])
            else:
                # controllo se non sono la stessa app ma diverse versioni
                
                new_df = new_df.append(tmp_df)
        else:
            new_df = new_df.append(tmp_df)
    return new_df

# Rimuovendo i duplicati rimangono presenti delle app più di una volta perchè sono presenti versione obsolete,
# La funzione controlla che tutti i campi corrispondano a meno di Last Updated e della Current ver
# Se r1.Last updated < r2.Last updated and r1.Current ver < r2.Current ver tengo la più aggiornata
def check_update_version(to_check):
    #controllo sempre che il n° di righe sia > 1 dopo groupby
    if(to_check.shape[0] != 1):
        
    else:
        return to_check

In [116]:
acc = remove_duplicate_rows(gps)
    


2033                  App Category  Rating  Reviews Size  Installs  Type  \
0     Coloring book moana   FAMILY     3.9      974  14M  500,000+  Free   

2033  Price Content Rating                     Genres      Last Updated  \
0         0       Everyone  Art & Design;Pretend Play  January 15, 2018   

2033 Current Ver   Android Ver     IntSize  IntInstalls Last Updated date  
0          2.0.0  4.0.3 and up  14000000.0       500000        2018-01-15  
2111                     App Category  Rating  Reviews  Size  Installs  Type  \
0     Mcqueen Coloring pages   FAMILY     NaN       65  7.0M  100,000+  Free   

2111  Price Content Rating                           Genres   Last Updated  \
0         0       Everyone  Art & Design;Action & Adventure  March 7, 2018   

2111 Current Ver Android Ver    IntSize  IntInstalls Last Updated date  
0          1.0.0  4.1 and up  7000000.0       100000        2018-03-07  
7027                                             App Category  Rating  \
0     U

4239                                          App       Category  Rating  \
0     Talkatone: Free Texts, Calls & Phone Number  COMMUNICATION     4.1   

4239  Reviews Size     Installs  Type  Price Content Rating         Genres  \
0      132015  25M  10,000,000+  Free      0       Everyone  Communication   

4239   Last Updated  Current Ver Android Ver     IntSize  IntInstalls  \
0     July 18, 2018          6.1  4.2 and up  25000000.0     10000000   

4239 Last Updated date  
0           2018-07-18  
389                        App       Category  Rating  Reviews  \
0    free video calls and chat  COMMUNICATION     4.2   594728   

389                Size     Installs  Type  Price Content Rating  \
0    Varies with device  50,000,000+  Free      0       Everyone   

389         Genres   Last Updated  Current Ver Android Ver  IntSize  \
0    Communication  July 19, 2018  7.3(800241)  4.4 and up      NaN   

389  IntInstalls Last Updated date  
0       50000000        2018-07-19  
5609  

514                 App Category  Rating  Reviews  Size  Installs  Type  \
0    FarmersOnly Dating   DATING     3.0     1146  1.4M  100,000+  Free   

514  Price Content Rating  Genres       Last Updated  Current Ver Android Ver  \
0        0     Mature 17+  Dating  February 25, 2016          2.2  4.0 and up   

514    IntSize  IntInstalls Last Updated date  
0    1000000.0       100000        2016-02-25  
568                                          App Category  Rating  Reviews  \
0    Dating for 50 plus Mature Singles – FINALLY   DATING     4.6    13049   

568 Size  Installs  Type  Price Content Rating  Genres   Last Updated  \
0    13M  500,000+  Free      0     Mature 17+  Dating  July 31, 2018   

568 Current Ver Android Ver     IntSize  IntInstalls Last Updated date  
0        4.17.2  4.1 and up  13000000.0       500000        2018-07-31  
576                            App Category  Rating  Reviews Size  Installs  \
0    Christian Dating For Free App   DATING     3.9     8723 

755            App   Category  Rating  Reviews  Size    Installs  Type  Price  \
0    C Programming  EDUCATION     4.3    22251  1.8M  1,000,000+  Free      0   

755 Content Rating     Genres   Last Updated  Current Ver Android Ver  \
0         Everyone  Education  June 28, 2015          3.0  2.3 and up   

755    IntSize  IntInstalls Last Updated date  
0    1000000.0      1000000        2015-06-28  
758        App   Category  Rating  Reviews  Size    Installs  Type  Price  \
0    Learn C++  EDUCATION     4.6    73404  5.3M  1,000,000+  Free      0   

758 Content Rating     Genres       Last Updated Current Ver Android Ver  \
0         Everyone  Education  December 25, 2017       4.5.2  4.0 and up   

758    IntSize  IntInstalls Last Updated date  
0    5000000.0      1000000        2017-12-25  
801                                                App   Category  Rating  \
0    Rosetta Stone: Learn to Speak & Read New Langu...  EDUCATION     4.5   

801  Reviews Size    Installs  Type

0     13000000.0       500000        2018-07-18  
9779                                            App Category  Rating  Reviews  \
0     The NBC App - Watch Live TV and Full Episodes   FAMILY     4.1    58104   

9779 Size    Installs  Type  Price Content Rating         Genres  \
0     17M  5,000,000+  Free      0           Teen  Entertainment   

9779  Last Updated Current Ver Android Ver     IntSize  IntInstalls  \
0     July 5, 2018      4.18.1  4.1 and up  17000000.0      5000000   

9779 Last Updated date  
0           2018-07-05  
945  App       Category  Rating  Reviews Size     Installs  Type  Price  \
0    WWE  ENTERTAINMENT     4.5   736864  20M  10,000,000+  Free      0   

945 Content Rating         Genres      Last Updated Current Ver Android Ver  \
0             Teen  Entertainment  January 19, 2018      3.17.2  4.1 and up   

945     IntSize  IntInstalls Last Updated date  
0    20000000.0     10000000        2018-01-19  
7606       App Category  Rating  Reviews  Size   

...

7446     App  Category  Rating  Reviews Size     Installs  Type  Price  \
0     CJmall  SHOPPING     3.7    18253  10M  10,000,000+  Free      0   

7446 Content Rating    Genres    Last Updated Current Ver Android Ver  \
0          Everyone  Shopping  August 6, 2018       6.3.8  4.1 and up   

7446     IntSize  IntInstalls Last Updated date  
0     10000000.0     10000000        2018-08-06  
                           App  Category  Rating  Reviews Size     Installs  \
0  Target - now with Cartwheel  SHOPPING     4.1    68406  24M  10,000,000+   
1  Target - now with Cartwheel  SHOPPING     4.1    68406  24M  10,000,000+   

   Type  Price Content Rating    Genres    Last Updated        Current Ver  \
0  Free      0       Everyone  Shopping   July 25, 2018  6.25.0+1906001476   
1  Free      0       Everyone  Shopping  August 6, 2018  6.25.0+1906001476   

  Android Ver     IntSize  IntInstalls Last Updated date  
0  5.0 and up  24000000.0     10000000        2018-07-25  
1  5.0 and up

2944                                                App     Category  Rating  \
0     Candy Camera - selfie, beauty camera, photo ed...  PHOTOGRAPHY     4.4   

2944  Reviews                Size      Installs  Type  Price Content Rating  \
0     3368705  Varies with device  100,000,000+  Free      0       Everyone   

2944       Genres   Last Updated  Current Ver   Android Ver  IntSize  \
0     Photography  July 16, 2018         4.47  4.0.3 and up      NaN   

2944  IntInstalls Last Updated date  
0       100000000        2018-07-16  
            App     Category  Rating  Reviews Size     Installs  Type  Price  \
0  Photo Editor  PHOTOGRAPHY     4.1   114680  27M  10,000,000+  Free      0   
1  Photo Editor  PHOTOGRAPHY     4.3    42079  17M   1,000,000+  Free      0   

  Content Rating       Genres        Last Updated         Current Ver  \
0       Everyone  Photography   February 15, 2018                1.38   
1       Everyone  Photography  September 24, 2015  Varies with device   

                             App Category  Rating  Reviews Size    Installs  \
0  Premier League - Official App   SPORTS     4.3    63580  24M  5,000,000+   
1  Premier League - Official App   SPORTS     4.3    63782  24M  5,000,000+   

   Type  Price Content Rating  Genres    Last Updated Current Ver Android Ver  \
0  Free      0       Everyone  Sports   July 20, 2018       1.1.5  4.1 and up   
1  Free      0       Everyone  Sports  August 7, 2018       1.1.5  4.1 and up   

      IntSize  IntInstalls Last Updated date  
0  24000000.0      5000000        2018-07-20  
1  24000000.0      5000000        2018-08-07  
10188                  App Category  Rating  Reviews Size    Installs  Type  \
0      ESPN Fantasy Sports   SPORTS     4.0   176487  10M  5,000,000+  Free   

10188  Price Content Rating  Genres       Last Updated Current Ver  \
0          0       Everyone  Sports  November 21, 2017       5.3.0   

10188 Android Ver     IntSize  IntInstalls Last Updated date  
0      4.4 and

3404                     App         Category  Rating  Reviews Size  \
0     Beautiful Widgets Free  PERSONALIZATION     3.9    25037  14M   

3404    Installs  Type  Price Content Rating           Genres  \
0     1,000,000+  Free      0       Everyone  Personalization   

3404      Last Updated Current Ver Android Ver     IntSize  IntInstalls  \
0     October 20, 2016       5.7.6  2.3 and up  14000000.0      1000000   

3404 Last Updated date  
0           2016-10-20  
3405         App         Category  Rating  Reviews Size    Installs  Type  \
0     HD Widgets  PERSONALIZATION     4.3    58617  26M  1,000,000+  Paid   

3405  Price Content Rating           Genres      Last Updated Current Ver  \
0     $0.99       Everyone  Personalization  December 7, 2016       4.3.2   

3405 Android Ver     IntSize  IntInstalls Last Updated date  
0     4.4 and up  26000000.0      1000000        2016-12-07  
3450             App      Category  Rating  Reviews                Size  \
0     Microsoft 

10646                                                App            Category  \
0      Podcast App: Free & Offline Podcasts by Player FM  NEWS_AND_MAGAZINES   

10646  Rating  Reviews Size    Installs  Type  Price Content Rating  \
0         4.6    66407  19M  1,000,000+  Free      0           Teen   

10646            Genres   Last Updated Current Ver Android Ver     IntSize  \
0      News & Magazines  July 25, 2018    4.1.0.72  4.0 and up  19000000.0   

10646  IntInstalls Last Updated date  
0          1000000        2018-07-25  
3777                    App            Category  Rating  Reviews Size  \
0     NYTimes - Latest News  NEWS_AND_MAGAZINES     3.9    63647  23M   

3777     Installs  Type  Price Content Rating            Genres  \
0     10,000,000+  Free      0   Everyone 10+  News & Magazines   

3777    Last Updated Current Ver Android Ver     IntSize  IntInstalls  \
0     August 1, 2018      6.19.5  4.4 and up  23000000.0     10000000   

3777 Last Updated date  
0      

In [108]:
type(pd.to_datetime(acc['Last Updated'].iloc[0]).date()) #January 7, 2018


datetime.date

In [45]:
prova.loc[prova['Reviews'].argmax()]

C:\Users\mporc\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


App                         Facebook
Category                      SOCIAL
Rating                           4.1
Reviews                     78158306
Size              Varies with device
Installs              1,000,000,000+
Type                            Free
Price                              0
Content Rating                  Teen
Genres                        Social
Last Updated          August 3, 2018
Current Ver       Varies with device
Android Ver       Varies with device
IntSize                          NaN
IntInstalls               1000000000
Name: 2544, dtype: object

In [41]:
prova = gps[gps['App'] == 'Facebook']

In [ ]:
gps_ur = pd.read_csv('googleplaystore_user_reviews.csv', low_memory = False)
#gps_ur.info()
gps_ur.head()